# Game instructions
Consider the following board game: A game board has 12 spaces. The swine senses the Christmas spirit and manages to run away from home couple of weeks beforehand. Fortunately for it, the butcher is a bit of a drunkard and easily distracted. The swine starts on space 7, and a butcher on space 1. On each game turn a 6-sided die is rolled. On a result of 1 to 3, the swine moves that many spaces forward. On a result of 5 or 6, the butcher moves that many spaces forward. On result 4, both advance one space forward. The swine wins if it reaches the river at space 12 (the final roll does not have to be exact, moving past space 12 is OK). The butcher wins if he catches up with the swine (or moves past it).

What are the probabilities of winning for the swine and the butcher?

Your assignment is to create a mathematical or statistical model to find these probabilities, and implement the solution as a computer program in whatever language you like. You will present it during the interview and we will discuss it with you. 

Consider the following questions as well: 
- Can you make your model easily extendable for different initial conditions (board size and initial positions)?
- Pros and cons of the approach?
- Can you say something about how long the game takes (also under different initial conditions)?

In [1]:
import numpy as np
import pandas as pd
import itertools

pd.options.display.max_columns = 70

# Model

In [27]:
##### Initiate game parameters #####

board_size = 12
swine_start = 7
butcher_start = 1

In [28]:
##### Create transition matrix #####

states = list(itertools.product(range(butcher_start, board_size + 1), range(swine_start, board_size + 1))) # Get S1 x S2.
states = list(itertools.compress(states, [i[0] <= i[1] for i in states])) # Filter out invalid states.
tm = pd.DataFrame(np.zeros(shape = (len(states), len(states))), index=states, columns=states) # Initiate transition matrix.

# Loop over states to fill in transition probabilities.
for b, s in states:
    
    # If someone already won, set as absorbing state.
    if s == board_size:
        tm.loc[[(b, s)], [(b, board_size)]] += 1
        continue
    elif b == s:
        tm.loc[[(b, s)], [(b, s)]] += 1
        continue
    
    # Die outcome 1.
    tm.loc[[(b, s)], [(b, s+1)]] += 1/6
    
    # Die outcome 2.
    if s+2 > board_size:
        tm.loc[[(b, s)], [(b, board_size)]] += 1/6
    else:
        tm.loc[[(b, s)], [(b, s+2)]] += 1/6
    
    # Die outcome 3.
    if s+3 > board_size:
        tm.loc[[(b, s)], [(b, board_size)]] += 1/6
    else:
        tm.loc[[(b, s)], [(b, s+3)]] += 1/6
    
    # Die outcome 4.
    tm.loc[[(b, s)], [(b+1, s+1)]] += 1/6 

     # Die outcome 5.
    if b+5 > s:
        tm.loc[[(b, s)], [(s, s)]] += 1/6
    else:
        tm.loc[[(b, s)], [(b+5, s)]] += 1/6

    # Die outcome 6.
    if b+6 > s:
        tm.loc[[(b, s)], [(s, s)]] += 1/6
    else:
        tm.loc[[(b, s)], [(b+6, s)]] += 1/6

In [43]:
##### Calculate final transition matrix #####

max_turns = (board_size - swine_start) + int((board_size - 2 - butcher_start)/5)
tm_final = pd.DataFrame(np.matrix(tm)**(max_turns), index=states, columns=states)

swine_win_prob = 0
butcher_win_prob = 0

for b, s in states:
    if b == s:
        butcher_win_prob += tm_final.get_value((butcher_start, swine_start), (b, s))
    else:
        swine_win_prob += tm_final.get_value((butcher_start, swine_start), (b, s))

In [44]:
print("If \n- the board size is {}, \n- the swine starts at position {}, \n- and the butcher starts at position {}, \nthen the swine's chance of escaping is {:.1f}% and the catch rate is {:.1f}%.".format(
    board_size, swine_start, butcher_start, swine_win_prob*100, butcher_win_prob*100))

If 
- the board size is 12, 
- the swine starts at position 7, 
- and the butcher starts at position 1, 
then the swine's chance of escaping is 51.2% and the catch rate is 48.8%.


In [52]:
##### Length of the game #####

game_length = np.zeros(shape = (max_turns))
tm_temp = tm.copy(deep = True)

for t in range(1, max_turns):
    tm_temp = pd.DataFrame(np.matrix(tm_temp)**t, index=states, columns=states)
    
    for b, s in states:
        if b == s:
            game_length[t] += tm_temp.get_value((butcher_start, swine_start), (b, s))
        elif s == board_size:
            game_length[t] += tm_temp.get_value((butcher_start, swine_start), (b, s))
    
    print(t5)
    print(tm_temp.loc[[(1,7)], ])
    print(game_length)        
    game_length[t] -= sum(game_length[:t])
    print(game_length)

1
        (1, 7)    (1, 8)    (1, 9)   (1, 10)  (1, 11)  (1, 12)  (2, 7)  \
(1, 7)     0.0  0.166667  0.166667  0.166667      0.0      0.0     0.0   

          (2, 8)  (2, 9)  (2, 10)  (2, 11)  (2, 12)  (3, 7)  (3, 8)  (3, 9)  \
(1, 7)  0.166667     0.0      0.0      0.0      0.0     0.0     0.0     0.0   

        (3, 10)  (3, 11)  (3, 12)  (4, 7)  (4, 8)  (4, 9)  (4, 10)  (4, 11)  \
(1, 7)      0.0      0.0      0.0     0.0     0.0     0.0      0.0      0.0   

        (4, 12)  (5, 7)  (5, 8)  (5, 9)  (5, 10)  (5, 11)  (5, 12)    (6, 7)  \
(1, 7)      0.0     0.0     0.0     0.0      0.0      0.0      0.0  0.166667   

        (6, 8)  (6, 9)  (6, 10)  (6, 11)  (6, 12)    (7, 7)  (7, 8)  (7, 9)  \
(1, 7)     0.0     0.0      0.0      0.0      0.0  0.166667     0.0     0.0   

        (7, 10)  (7, 11)  (7, 12)  (8, 8)  (8, 9)  (8, 10)  (8, 11)  (8, 12)  \
(1, 7)      0.0      0.0      0.0     0.0     0.0      0.0      0.0      0.0   

        (9, 9)  (9, 10)  (9, 11)  (9, 12)  (10, 10

In [46]:
game_length

array([ 0.        ,  0.16666667,  0.16666667,  0.83333333,  0.16666667,
        0.83333333])

In [35]:
game_length[t] -= game_length[t-1]

In [6]:









# tm_swine = np.zeros(shape = (board_size, board_size + 4))
# np.fill_diagonal(tm_swine[swine_start-1:, swine_start-1:], 2/6)

# for i in range(swine_start-1, board_size):
#     tm_swine[i, i+1] = 2/6
#     tm_swine[i, i+2:i+4] = 1/6
    
# tm_swine = tm_swine[:, :-4]

# for i in range(swine_start-1, board_size):
#     if 1 - sum(tm_swine[i, :]) > 0.000001:
#         tm_swine[i, -1] += 1 - sum(tm_swine[i, :])
#     tm_swine[-1, -1] = 1
    

# tm_butcher = np.zeros(shape = (board_size, board_size + 6))
# np.fill_diagonal(tm_butcher[butcher_start-1:, butcher_start-1:], 3/6)

# for i in range(butcher_start-1, board_size):
#     tm_butcher[i, i+1] = 1/6
#     tm_butcher[i, i+5:i+7] = 1/6
    
# tm_butcher = tm_butcher[:, :-6]

# for i in range(butcher_start-1, board_size):
#     if 1 - sum(tm_butcher[i, :]) > 0.000001:
#         tm_butcher[i, -1] += 1 - sum(tm_butcher[i, :])
#     tm_butcher[-1, -1] = 1